In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt

# 读取本地 data 文件夹的 BTC/USDT 1d 数据
df = pd.read_csv('data/spot/binance/1d/BTC_USDT.csv')
df['date'] = pd.to_datetime(df['datetime'])
df.set_index('date', inplace=True)

# 计算连续涨跌天数
close = df['close']
ret = close.pct_change()
up = ret > 0
down = ret < 0

up_streak = up.rolling(3).sum() == 3
down_streak = down.rolling(3).sum() == 3

# 计算3天涨跌幅
change_3d = close.pct_change(3).abs()

# 生成信号
entries = up_streak.shift(1).fillna(False)  # 连涨3天后第4天开多
exits = down_streak.shift(1).fillna(False)  # 连跌3天后第4天开空

# 止盈止损幅度
take_profit = change_3d.shift(1).fillna(0)
stop_loss = change_3d.shift(1).fillna(0)

# 用vectorbt回测
pf = vbt.Portfolio.from_signals(
    close,
    entries,
    exits,
    sl_stop=stop_loss,      # 止损百分比
    tp_stop=take_profit,    # 止盈百分比
    direction='both',       # 多空都做
    freq='1D'
)

print(pf.stats())
# 将回测结果的交互式图表保存为HTML文件，便于浏览器打开
fig = pf.plot()
fig.write_html("portfolio_result.html")
print("已保存为 portfolio_result.html，可用浏览器打开查看。")

Start                               2017-08-17 00:00:00
End                                 2025-09-07 00:00:00
Period                               2944 days 00:00:00
Start Value                                       100.0
End Value                                    178.222659
Total Return [%]                              78.222659
Benchmark Return [%]                        2493.588451
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              66.138006
Max Drawdown Duration                1500 days 00:00:00
Total Trades                                        284
Total Closed Trades                                 283
Total Open Trades                                     1
Open Trade PnL                                      0.0
Win Rate [%]                                   47.70318
Best Trade [%]                                45.598031
Worst Trade [%]                              -25

In [ ]:
import ccxt
import pandas as pd
import datetime

# 步骤1：用ccxt获取Binance BTC/USDT 1d数据

exchange = ccxt.binance()
since = exchange.parse8601('2020-01-01T00:00:00Z')
bars = []
limit = 1000
symbol = 'BTC/USDT'
timeframe = '1d'
while True:
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
    if not ohlcv:
        break
    bars += ohlcv
    since = ohlcv[-1][0] + 24*60*60*1000  # 下一天
    if len(ohlcv) < limit:
        break

df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('date', inplace=True)